# Real-Time Stream Viewer (HTTP)
the following function responds to HTTP requests with the list of last 10 processed twitter messages + sentiments in reverse order (newest on top), it reads records from the enriched stream, take the recent 10 messages, and reverse sort them. the function is using nuclio context to store the last results and stream pointers for max efficiency.<br> 

The code is automatically converted into a nuclio (serverless) function and and respond to HTTP requests<br>

the example demonstrate the use of `%nuclio` magic commands to specify environment variables, package dependencies,<br>configurations, and to deploy functions automatically onto a cluster.


## Initialize nuclio emulation, environment variables and configuration
use `# nuclio: ignore` for sections that don't need to be copied to the function

In [1]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

In [2]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

### Set function configuration 
use a cron trigger with 5min interval and define the base image<br>
for more details check [nuclio function configuration reference](https://github.com/nuclio/nuclio/blob/master/docs/reference/function-configuration/function-configuration-reference.md)

In [3]:
%%nuclio config 
# ask for a specific (fixed) API port
spec.triggers.web.kind = "http"
spec.triggers.web.attributes.port = 30100
# define the function base docker image
spec.build.baseImage = "mlrun/mlrun"

%nuclio: setting spec.triggers.web.kind to 'http'
%nuclio: setting spec.triggers.web.attributes.port to 30100
%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'


### Install required packages
`%nuclio cmd` allows you to run image build instructions and install packages<br>
Note: `-c` option will only install in nuclio, not locally

In [4]:
%nuclio cmd -c pip install v3io

## Nuclio function implementation
this function can run in Jupyter or in nuclio (real-time serverless)

In [5]:
import v3io.dataplane
import json
import os

def init_context(context):
    access_key = os.getenv('V3IO_ACCESS_KEY', None)
    setattr(context, 'container', os.getenv('V3IO_CONTAINER', 'bigdata'))
    setattr(context, 'stream_path', os.getenv('STOCKS_STREAM', 'stocks/stocks_stream'))
    
    v3io_client = v3io.dataplane.Client(endpoint=os.getenv('V3IO_API', None), access_key=access_key)
    resp = v3io_client.seek_shard(container=context.container, path=f'{context.stream_path}/0', seek_type='EARLIEST')
    setattr(context, 'next_location', resp.output.location)
    setattr(context, 'data', [])
    setattr(context, 'v3io_client', v3io_client) 
    setattr(context, 'limit', os.getenv('LIMIT', 10))

    
def handler(context, event):
    
    resp = context.v3io_client.get_records(container=context.container, path=f'{context.stream_path}/0', location=context.next_location, limit=context.limit)
    context.next_location = resp.output.next_location
    context.logger.info('location: %s', context.next_location)

    for rec in resp.output.records:
        rec_data = rec.data.decode('utf-8')
        rec_json = json.loads(rec_data)
        context.data.append(rec_json)

    context.data = context.data[-context.limit:]

    return json.dumps(context.data[::-1])
                            

In [6]:
# nuclio: end-code

## Function invocation
the following section simulates nuclio function invocation and will emit the function results

In [7]:
# create a test event and invoke the function locally
init_context(context)
event = nuclio.Event(body='')
handler(context, event)

Python> 2020-09-15 21:25:08,121 [info] location: AQAAAGYAAAAAAEB8IAAAAA==


'[{"content": "By Yasin Ebrahim\\nInvesting.com \\u2013 The Dow ended roughly flat, pressured by a fall in financials and u-turn in Apple after the tech giant launched a new services bundle and hardware.   \\nThe Dow Jones Industrial Average ended flat or 1 point higher. The S&P 500 was up 0.49%, while the Nasdaq Composite added 1.21%.\\nApple  unveiled Tuesday a slate of new hardware including a new iPad and Apple Watch and launched its much-rumored services bundle, Apple One, starting at $14.95 per month. Apple rose 0.2%, but had been up more than 1% in early trade.\\nThe bundle will be launched in three tiers: individual, family, and premier offering. Each will include services such as Apple Music, Apple TV+, Apple Arcade, iCloud storage, the company said. Apple News+ and Apple Fitness+, however, are included with the more expensive premium subscription at $29.95 a month, the company said. \\nThe virtual product event comes ahead of the widely anticipated launch of the iPhone 12, ex

## Deploy a function onto a cluster
the `%nuclio deploy` command deploy functions into a cluster, make sure the notebook is saved prior to running it !<br>check the help (`%nuclio help deploy`) for more information

In [8]:
from mlrun import code_to_function

environment_variables = {'V3IO_CONTAINER': 'bigdata',
                         'STOCKS_STREAM': 'stocks/stocks_stream'}

fn = code_to_function('stream-viewer',
                      kind='nuclio',
                      handler='handler')
fn.export('05-stream-viewer.yaml')
fn.set_envs(environment_variables)

> 2020-09-15 21:25:11,955 [info] function spec saved to path: 05-stream-viewer.yaml


In [9]:
fn.deploy(project='stocks')

> 2020-09-15 21:25:12,667 [info] deploy started
[nuclio] 2020-09-15 21:25:14,802 (info) Build complete
[nuclio] 2020-09-15 21:25:18,853 (info) Function deploy complete
[nuclio] 2020-09-15 21:25:18,863 done updating stocks-stream-viewer, function address: 192.168.224.209:30100


'http://192.168.224.209:30100'

In [10]:
# nuclio: ignore
# test the new API end point, take the address from the deploy log above
!curl 192.168.224.209:30100

[{"content": "By Yasin Ebrahim\nInvesting.com \u2013 The Dow ended roughly flat, pressured by a fall in financials and u-turn in Apple after the tech giant launched a new services bundle and hardware.   \nThe Dow Jones Industrial Average ended flat or 1 point higher. The S&P 500 was up 0.49%, while the Nasdaq Composite added 1.21%.\nApple  unveiled Tuesday a slate of new hardware including a new iPad and Apple Watch and launched its much-rumored services bundle, Apple One, starting at $14.95 per month. Apple rose 0.2%, but had been up more than 1% in early trade.\nThe bundle will be launched in three tiers: individual, family, and premier offering. Each will include services such as Apple Music, Apple TV+, Apple Arcade, iCloud storage, the company said. Apple News+ and Apple Fitness+, however, are included with the more expensive premium subscription at $29.95 a month, the company said. \nThe virtual product event comes ahead of the widely anticipated launch of the iPhone 12, expected 